In [1]:
import Pkg

# coment out once pkgs are loaded

Pkg.add("CSV")
Pkg.add("EzXML")
Pkg.add("DataFrames")
Pkg.add("BenchmarkTools")
Pkg.add("GLM")
Pkg.add("Distributions")
Pkg.add("Plots")

   Resolving package versions...
  No Changes to `C:\Users\NizarUribe\.julia\environments\v1.11\Project.toml`
  No Changes to `C:\Users\NizarUribe\.julia\environments\v1.11\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\NizarUribe\.julia\environments\v1.11\Project.toml`
  No Changes to `C:\Users\NizarUribe\.julia\environments\v1.11\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\NizarUribe\.julia\environments\v1.11\Project.toml`
  No Changes to `C:\Users\NizarUribe\.julia\environments\v1.11\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\NizarUribe\.julia\environments\v1.11\Project.toml`
  No Changes to `C:\Users\NizarUribe\.julia\environments\v1.11\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\NizarUribe\.julia\environments\v1.11\Project.toml`
  No Changes to `C:\Users\NizarUribe\.julia\environments\v1.11\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\N

In [3]:
Pkg.add("Distributed")
Pkg.add("Dates")
Pkg.add("DataFrames")

   Resolving package versions...
  No Changes to `C:\Users\NizarUribe\.julia\environments\v1.11\Project.toml`
  No Changes to `C:\Users\NizarUribe\.julia\environments\v1.11\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\NizarUribe\.julia\environments\v1.11\Project.toml`
  No Changes to `C:\Users\NizarUribe\.julia\environments\v1.11\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\NizarUribe\.julia\environments\v1.11\Project.toml`
  No Changes to `C:\Users\NizarUribe\.julia\environments\v1.11\Manifest.toml`


In [4]:
using Distributed
using CSV
using DataFrames
using Dates

# Inicia el temporizador
start_time = now()

# Carga los archivos CSV
ratings = CSV.read("ratings_ej.csv", DataFrame)
movies = CSV.read("movies_ej.csv", DataFrame)

# Dividir el dataset para procesamiento concurrente
addprocs()  # Añadir procesos paralelos (puedes especificar el número si es necesario)

@everywhere function count_ratings_by_genre(ratings, movies)
    # Unir las tablas por 'movieId'
    data = innerjoin(ratings, movies, on=:movieId)
    
    # Inicializar un diccionario para contar las calificaciones por género
    genre_count = Dict{String, Int}()
    
    for row in eachrow(data)
        genres = split(row.genres, "|")  # Dividir los géneros por '|'
        for genre in genres
            genre_count[genre] = get(genre_count, genre, 0) + 1
        end
    end
    return genre_count
end

# Map Reduce en paralelo
@distributed (+) for i in 1:nworkers()
    partial_result = count_ratings_by_genre(ratings, movies)
end

# Reducir los resultados
final_result = reduce((x, y) -> mergewith(+, x, y), partial_result)

# Mostrar el tiempo total de ejecución
end_time = now()
total_time = end_time - start_time
println("Tiempo total de ejecución: ", total_time)

# Imprimir los resultados
for (genre, count) in final_result
    println("$genre: $count calificaciones")
end


TaskFailedException: TaskFailedException

    nested task error: On worker 2:
    KeyError: key Base.PkgId(Base.UUID("a93c6f00-e57d-5684-b7b6-d8193f3e46c0"), "DataFrames") not found
    Stacktrace:
      [1] getindex
        @ .\dict.jl:477 [inlined]
      [2] macro expansion
        @ .\lock.jl:273 [inlined]
      [3] root_module
        @ .\loading.jl:2376
      [4] deserialize_module
        @ C:\Users\NizarUribe\.julia\juliaup\julia-1.11.0+0.x64.w64.mingw32\share\julia\stdlib\v1.11\Serialization\src\Serialization.jl:1024
      [5] handle_deserialize
        @ C:\Users\NizarUribe\.julia\juliaup\julia-1.11.0+0.x64.w64.mingw32\share\julia\stdlib\v1.11\Serialization\src\Serialization.jl:926
      [6] deserialize
        @ C:\Users\NizarUribe\.julia\juliaup\julia-1.11.0+0.x64.w64.mingw32\share\julia\stdlib\v1.11\Serialization\src\Serialization.jl:844
      [7] deserialize_datatype
        @ C:\Users\NizarUribe\.julia\juliaup\julia-1.11.0+0.x64.w64.mingw32\share\julia\stdlib\v1.11\Serial

In [5]:
using Distributed
@everywhere using CSV
@everywhere using DataFrames
@everywhere using Dates

# Inicia el temporizador
start_time = now()

# Carga los archivos CSV solo en el nodo maestro
ratings = CSV.read("ratings_ej.csv", DataFrame)
movies = CSV.read("movies_ej.csv", DataFrame)

# Enviar los datasets a todos los nodos
@everywhere ratings = CSV.read("ratings_ej.csv", DataFrame)
@everywhere movies = CSV.read("movies_ej.csv", DataFrame)

# Función para contar las calificaciones por género
@everywhere function count_ratings_by_genre(ratings, movies)
    # Unir las tablas por 'movieId'
    data = innerjoin(ratings, movies, on=:movieId)
    
    # Inicializar un diccionario para contar las calificaciones por género
    genre_count = Dict{String, Int}()
    
    for row in eachrow(data)
        genres = split(row.genres, "|")  # Dividir los géneros por '|'
        for genre in genres
            genre_count[genre] = get(genre_count, genre, 0) + 1
        end
    end
    return genre_count
end

# Añadir procesos paralelos
addprocs()

# Ejecutar el conteo de manera distribuida
partial_results = @distributed (mergewith(+, _ , _)) for i in 1:nworkers()
    count_ratings_by_genre(ratings, movies)
end

# Reducir los resultados
final_result = reduce((x, y) -> mergewith(+, x, y), partial_results)

# Mostrar el tiempo total de ejecución
end_time = now()
total_time = end_time - start_time
println("Tiempo total de ejecución: ", total_time)

# Imprimir los resultados
for (genre, count) in final_result
    println("$genre: $count calificaciones")
end


ErrorException: syntax: all-underscore identifiers are write-only and their values cannot be used in expressions

In [6]:
using Distributed
@everywhere using CSV
@everywhere using DataFrames
@everywhere using Dates

# Inicia el temporizador
start_time = now()

# Carga los archivos CSV solo en el nodo maestro
ratings = CSV.read("ratings_ej.csv", DataFrame)
movies = CSV.read("movies_ej.csv", DataFrame)

# Enviar los datasets a todos los nodos
@everywhere ratings = CSV.read("ratings_ej.csv", DataFrame)
@everywhere movies = CSV.read("movies_ej.csv", DataFrame)

# Función para contar las calificaciones por género
@everywhere function count_ratings_by_genre(ratings, movies)
    # Unir las tablas por 'movieId'
    data = innerjoin(ratings, movies, on=:movieId)
    
    # Inicializar un diccionario para contar las calificaciones por género
    genre_count = Dict{String, Int}()
    
    for row in eachrow(data)
        genres = split(row.genres, "|")  # Dividir los géneros por '|'
        for genre in genres
            genre_count[genre] = get(genre_count, genre, 0) + 1
        end
    end
    return genre_count
end

# Añadir procesos paralelos
addprocs()

# Ejecutar el conteo de manera distribuida
partial_results = @distributed (mergewith(+, a , b)) for i in 1:nworkers()
    count_ratings_by_genre(ratings, movies)
end

# Reducir los resultados
final_result = reduce((x, y) -> mergewith(+, x, y), partial_results)

# Mostrar el tiempo total de ejecución
end_time = now()
total_time = end_time - start_time
println("Tiempo total de ejecución: ", total_time)

# Imprimir los resultados
for (genre, count) in final_result
    println("$genre: $count calificaciones")
end


UndefVarError: UndefVarError: `a` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [7]:
using Distributed
@everywhere using CSV
@everywhere using DataFrames
@everywhere using Dates

# Inicia el temporizador
start_time = now()

# Carga los archivos CSV solo en el nodo maestro
ratings = CSV.read("ratings_ej.csv", DataFrame)
movies = CSV.read("movies_ej.csv", DataFrame)

# Enviar los datasets a todos los nodos
@everywhere ratings = CSV.read("ratings_ej.csv", DataFrame)
@everywhere movies = CSV.read("movies_ej.csv", DataFrame)

# Función para contar las calificaciones por género
@everywhere function count_ratings_by_genre(ratings, movies)
    # Unir las tablas por 'movieId'
    data = innerjoin(ratings, movies, on=:movieId)
    
    # Inicializar un diccionario para contar las calificaciones por género
    genre_count = Dict{String, Int}()
    
    for row in eachrow(data)
        genres = split(row.genres, "|")  # Dividir los géneros por '|'
        for genre in genres
            genre_count[genre] = get(genre_count, genre, 0) + 1
        end
    end
    return genre_count
end

# Añadir procesos paralelos
addprocs()

# Ejecutar el conteo de manera distribuida
partial_results = @distributed (mergewith(+, count_ratings_by_genre(ratings, movies), count_ratings_by_genre(ratings, movies))) for i in 1:nworkers()
end

# Reducir los resultados
final_result = reduce((x, y) -> mergewith(+, x, y), partial_results)

# Mostrar el tiempo total de ejecución
end_time = now()
total_time = end_time - start_time
println("Tiempo total de ejecución: ", total_time)

# Imprimir los resultados
for (genre, count) in final_result
    println("$genre: $count calificaciones")
end


MethodError: MethodError: objects of type Dict{String, Int64} are not callable
The object of type `Dict{String, Int64}` exists, but no method is defined for this combination of argument types when trying to treat it as a callable object.

In [9]:
using Distributed
@everywhere using CSV
@everywhere using DataFrames
@everywhere using Dates

# Inicia el temporizador
start_time = now()

# Carga los archivos CSV solo en el nodo maestro
ratings = CSV.read("ratings_ej.csv", DataFrame)
movies = CSV.read("movies_ej.csv", DataFrame)

# Enviar los datasets a todos los nodos
@everywhere ratings = CSV.read("ratings_ej.csv", DataFrame)
@everywhere movies = CSV.read("movies_ej.csv", DataFrame)

# Función para contar las calificaciones por género
@everywhere function count_ratings_by_genre(ratings, movies)
    # Unir las tablas por 'movieId'
    data = innerjoin(ratings, movies, on=:movieId)
    
    # Inicializar un diccionario para contar las calificaciones por género
    genre_count = Dict{String, Int}()
    
    for row in eachrow(data)
        genres = split(row.genres, "|")  # Dividir los géneros por '|'
        for genre in genres
            genre_count[genre] = get(genre_count, genre, 0) + 1
        end
    end
    return genre_count
end

# Añadir procesos paralelos
addprocs()

# Ejecutar el conteo de manera distribuida
partial_results = @distributed (vcat) for i in 1:nworkers()
    count_ratings_by_genre(ratings, movies)
end

# Reducir los resultados parciales utilizando mergewith para combinar los diccionarios
final_result = reduce((x, y) -> mergewith(+, x, y), partial_results)

# Mostrar el tiempo total de ejecución
end_time = now()
total_time = end_time - start_time
println("Tiempo total de ejecución: ", total_time)

# Imprimir los resultados
for (genre, count) in final_result
    println("$genre: $count calificaciones")
end


TaskFailedException: TaskFailedException

    nested task error: IOError: write: no buffer space available (ENOBUFS)
    Stacktrace:
      [1] uv_write(s::Sockets.TCPSocket, p::Ptr{UInt8}, n::UInt64)
        @ Base .\stream.jl:1091
      [2] unsafe_write(s::Sockets.TCPSocket, p::Ptr{UInt8}, n::UInt64)
        @ Base .\stream.jl:1145
      [3] unsafe_write
        @ .\io.jl:803 [inlined]
      [4] write
        @ .\io.jl:837 [inlined]
      [5] serialize_array_data
        @ C:\Users\NizarUribe\.julia\juliaup\julia-1.11.0+0.x64.w64.mingw32\share\julia\stdlib\v1.11\Serialization\src\Serialization.jl:254 [inlined]
      [6] serialize(s::Distributed.ClusterSerializer{Sockets.TCPSocket}, a::Vector{Float64})
        @ Serialization C:\Users\NizarUribe\.julia\juliaup\julia-1.11.0+0.x64.w64.mingw32\share\julia\stdlib\v1.11\Serialization\src\Serialization.jl:271
      [7] serialize(s::Distributed.ClusterSerializer{Sockets.TCPSocket}, a::Vector{AbstractVector})
        @ Serialization C:\Users\N

In [1]:
using CSV
using DataFrames
using Dates

# Inicia el temporizador
start_time = now()

# Carga los archivos CSV en el nodo principal
ratings = CSV.read("ratings_ej.csv", DataFrame)
movies = CSV.read("movies_ej.csv", DataFrame)

# Función para contar las calificaciones por género
function count_ratings_by_genre(ratings, movies)
    # Unir las tablas por 'movieId'
    data = innerjoin(ratings, movies, on=:movieId)
    
    # Inicializar un diccionario para contar las calificaciones por género
    genre_count = Dict{String, Int}()
    
    for row in eachrow(data)
        genres = split(row.genres, "|")  # Dividir los géneros por '|'
        for genre in genres
            genre_count[genre] = get(genre_count, genre, 0) + 1
        end
    end
    return genre_count
end

# Ejecutar el conteo de calificaciones por género
final_result = count_ratings_by_genre(ratings, movies)

# Mostrar el tiempo total de ejecución
end_time = now()
total_time = end_time - start_time
println("Tiempo total de ejecución: ", total_time)

# Imprimir los resultados
for (genre, count) in final_result
    println("$genre: $count calificaciones")
end


Tiempo total de ejecución: 4397 milliseconds
War: 52955 calificaciones
Documentary: 12337 calificaciones
Action: 312990 calificaciones
Adventure: 247048 calificaciones
Mystery: 82845 calificaciones
Children: 92544 calificaciones
Thriller: 282542 calificaciones
Sci-Fi: 181124 calificaciones
Comedy: 377729 calificaciones
(no genres listed): 960 calificaciones
IMAX: 44760 calificaciones
Fantasy: 119778 calificaciones
Drama: 456202 calificaciones
Animation: 69941 calificaciones
Western: 20430 calificaciones
Horror: 80261 calificaciones
Romance: 190849 calificaciones
Film-Noir: 9785 calificaciones
Crime: 173731 calificaciones
Musical: 40483 calificaciones


In [3]:
using CSV
using DataFrames
using Dates

# Inicia el temporizador
start_time = now()

# Carga los archivos CSV en el nodo principal
ratings = CSV.read("split-ratings-ij_ej.csv", DataFrame)
movies = CSV.read("movies_ej.csv", DataFrame)

# Función para contar las calificaciones por género
function count_ratings_by_genre(ratings, movies)
    # Unir las tablas por 'movieId'
    data = innerjoin(ratings, movies, on=:movieId)
    
    # Inicializar un diccionario para contar las calificaciones por género
    genre_count = Dict{String, Int}()
    
    for row in eachrow(data)
        genres = split(row.genres, "|")  # Dividir los géneros por '|'
        for genre in genres
            genre_count[genre] = get(genre_count, genre, 0) + 1
        end
    end
    return genre_count
end

# Ejecutar el conteo de calificaciones por género
final_result = count_ratings_by_genre(ratings, movies)

# Mostrar el tiempo total de ejecución
end_time = now()
total_time = end_time - start_time
println("Tiempo total de ejecución: ", total_time)

# Imprimir los resultados
for (genre, count) in final_result
    println("$genre: $count calificaciones")
end

Tiempo total de ejecución: 139 milliseconds
War: 4897 calificaciones
Documentary: 1355 calificaciones
Action: 31138 calificaciones
Adventure: 23746 calificaciones
Mystery: 8209 calificaciones
Children: 8292 calificaciones
Thriller: 27883 calificaciones
Sci-Fi: 18025 calificaciones
Comedy: 34412 calificaciones
(no genres listed): 103 calificaciones
IMAX: 4475 calificaciones
Fantasy: 11207 calificaciones
Drama: 44034 calificaciones
Animation: 6473 calificaciones
Western: 1841 calificaciones
Horror: 7713 calificaciones
Romance: 17456 calificaciones
Film-Noir: 904 calificaciones
Crime: 17175 calificaciones
Musical: 3527 calificaciones


# Sábado 26 de octubre, 2024

In [ ]:
using CSV
using DataFrames
using Dates
using Threads

# Función para cargar todos los archivos que comienzan con "HEADER"
function load_ratings_files(directory)
    files = readdir(directory)
    ratings_files = filter(x -> startswith(x, "HEADER"), files)
    
    ratings_data = DataFrame[]
    for file in ratings_files
        push!(ratings_data, CSV.read(joinpath(directory, file), DataFrame))
    end
    return vcat(ratings_data...)  # Concatenar todos los DataFrames
end

# Función para contar las calificaciones por género en paralelo
function count_ratings_by_genre_threaded(ratings, movies)
    # Unir las tablas por 'movieId'
    data = innerjoin(ratings, movies, on=:movieId)

    # Inicializar un diccionario compartido para contar las calificaciones por género
    genre_count = Dict{String, Int}()
    lock = ReentrantLock()  # Lock para proteger el acceso concurrente al diccionario
    
    Threads.@threads for row in eachrow(data)
        genres = split(row.genres, "|")  # Dividir los géneros por '|'
        for genre in genres
            lock(lock) do
                genre_count[genre] = get(genre_count, genre, 0) + 1
            end
        end
    end
    return genre_count
end

# Inicia el temporizador
start_time = now()

# Carga los archivos CSV en el nodo principal
ratings = load_ratings_files(".")  # Cambia el "." por el directorio donde están los archivos "HEADER"
movies = CSV.read("movies_ej.csv", DataFrame)

# Ejecutar el conteo de calificaciones por género en paralelo
final_result = count_ratings_by_genre_threaded(ratings, movies)

# Mostrar el tiempo total de ejecución
end_time = now()
total_time = end_time - start_time
println("Tiempo total de ejecución: ", total_time)

# Imprimir los resultados
for (genre, count) in final_result
    println("$genre: $count calificaciones")
end


In [7]:
#Dividir archivo Martha Rico

using Pkg
Pkg.add("CSV")
Pkg.add("DataFrames")


using CSV
using DataFrames


# Cargar el archivo CSV
df = CSV.File("ratings.csv") |> DataFrame


# Definir el número de partes en las que quieres dividir el archivo
N = 10


# Calcular el tamaño de cada parte
num_rows = nrow(df)
rows_per_part = div(num_rows, N)


# Crear un loop para dividir el DataFrame y guardar cada parte
for i in 0:(N-1)
    start_row = i * rows_per_part + 1
    end_row = (i == N - 1) ? num_rows : (start_row + rows_per_part - 1)


    # Crear un DataFrame para la parte actual
    part_df = df[start_row:end_row, :]


    # Guardar la parte como un nuevo archivo CSV
    CSV.write("ratings_part_$(i+1).csv", part_df)
end


println("El archivo ha sido dividido en $N partes.")


   Resolving package versions...
  No Changes to `C:\Users\NizarUribe\.julia\environments\v1.11\Project.toml`
  No Changes to `C:\Users\NizarUribe\.julia\environments\v1.11\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\NizarUribe\.julia\environments\v1.11\Project.toml`
  No Changes to `C:\Users\NizarUribe\.julia\environments\v1.11\Manifest.toml`


El archivo ha sido dividido en 10 partes.


In [8]:
using CSV
using DataFrames
using Base.Threads
using Printf

# Asegúrate de que el número de hilos esté configurado
println("Número de hilos disponibles: ", nthreads())

# Función para cargar los archivos CSV y contar calificaciones por género
function cargar_y_contar(i, movies)
    ratings = CSV.read("ratings_part_" * string(i) * ".csv", DataFrame)
    return count_ratings_by_genre(ratings, movies)
end

# Función para contar las calificaciones por género
function count_ratings_by_genre(ratings, movies)
    # Unir las tablas por 'movieId'
    data = innerjoin(ratings, movies, on=:movieId)
   
    # Inicializar un diccionario para contar las calificaciones por género
    genre_count = Dict{String, Int}()
   
    for row in eachrow(data)
        genres = split(row.genres, "|")  # Dividir los géneros por '|'
        for genre in genres
            genre_count[genre] = get(genre_count, genre, 0) + 1
        end
    end
    return genre_count
end

# Cargar la lista de películas
movies = CSV.read("movies.csv", DataFrame)

# Arreglo para almacenar los resultados de cada hilo
results = Vector{Dict{String, Int}}(undef, 10)

# Usar Threads.@threads para ejecutar en paralelo
Threads.@threads for i in 1:10
    results[i] = cargar_y_contar(i, movies)
end

# Combinar los resultados de todos los hilos
final_result = Dict{String, Int}()
for result in results
    for (genre, count) in result
        final_result[genre] = get(final_result, genre, 0) + count
    end
end



function format_with_commas(n::Int)
    s = string(n)
    len = length(s)
    
    # Si la longitud es menor o igual a 3, solo retorna el número
    if len <= 3
        return s
    end
    
    # Crear una cadena con comas
    result = ""
    for (i, digit) in enumerate(reverse(s))
        if i > 1 && (i - 1) % 3 == 0
            result *= ","
        end
        result *= digit
    end
    
    return reverse(result)
end


# Iterar sobre el diccionario e imprimir el resultado formateado
for (genre, count) in final_result
    formated_count = format_with_commas(count)
    println("$genre: \t $formated_count calificaciones")
end

Número de hilos disponibles: 1
War: 	 1,267,346 calificaciones
Documentary: 	 322,449 calificaciones
Action: 	 7,446,918 calificaciones
Adventure: 	 5,832,424 calificaciones
Mystery: 	 2,010,995 calificaciones
Children: 	 2,124,258 calificaciones
Thriller: 	 6,763,272 calificaciones
Sci-Fi: 	 4,325,740 calificaciones
Comedy: 	 8,926,230 calificaciones
(no genres listed): 	 26,627 calificaciones
IMAX: 	 1,063,279 calificaciones
Fantasy: 	 2,831,585 calificaciones
Drama: 	 10,962,833 calificaciones
Animation: 	 1,630,987 calificaciones
Western: 	 483,731 calificaciones
Horror: 	 1,892,183 calificaciones
Romance: 	 4,497,291 calificaciones
Film-Noir: 	 247,227 calificaciones
Crime: 	 4,190,259 calificaciones
Musical: 	 964,252 calificaciones


In [9]:
using CSV
using DataFrames
using Distributed
using Printf

# Añadir los trabajadores para paralelismo
addprocs()  # O especifica el número de procesos, ej. addprocs(4)

@everywhere begin
    using CSV
    using DataFrames
end

# Función para cargar y contar calificaciones por género
@everywhere function cargar_y_contar(i, movies)
    ratings = CSV.read("ratings_part_" * string(i) * ".csv", DataFrame)
    return count_ratings_by_genre(ratings, movies)
end

@everywhere function count_ratings_by_genre(ratings, movies)
    data = innerjoin(ratings, movies, on=:movieId)
    genre_count = Dict{String, Int}()
    for row in eachrow(data)
        genres = split(row.genres, "|")
        for genre in genres
            genre_count[genre] = get(genre_count, genre, 0) + 1
        end
    end
    return genre_count
end

# Cargar la lista de películas en el proceso principal
movies = CSV.read("movies.csv", DataFrame)

# Ejecutar el conteo en paralelo usando @distributed
@distributed (merge) for i in 1:10
    cargar_y_contar(i, movies)
end

# Función para combinar los resultados de los procesos
final_result = @distributed (Dict{String, Int}) for i in 1:10
    results[i] = cargar_y_contar(i, movies)
end

function format_with_commas(n::Int)
    s = string(n)
    len = length(s)
    if len <= 3 return s end
    result = ""
    for (i, digit) in enumerate(reverse(s))
        if i > 1 && (i - 1) % 3 == 0
            result *= ","
        end
        result *= digit
    end
    return reverse(result)
end

# Imprimir el resultado
for (genre, count) in final_result
    formatted_count = format_with_commas(count)
    println("$genre: \t $formatted_count calificaciones")
end


      From worker 6:	Internal error: during type inference of
      From worker 6:	(::Distributed.var"#103#104"{Sockets.TCPSocket, Sockets.TCPSocket, Bool})()
      From worker 6:	Encountered unexpected error in runtime:
      From worker 6:	ReadOnlyMemoryError()
      From worker 6:	set_page_metadata at C:/workdir/src\gc.h:413 [inlined]
      From worker 6:	gc_add_page at C:/workdir/src\gc.c:1305
      From worker 6:	jl_gc_pool_alloc_inner at C:/workdir/src\gc.c:1363
      From worker 6:	ijl_gc_pool_alloc_instrumented at C:/workdir/src\gc.c:1383
      From worker 6:	argextype at .\compiler\optimize.jl:425
      From worker 6:	argextype at .\compiler\optimize.jl:422 [inlined]
      From worker 6:	argextype at .\compiler\optimize.jl:421 [inlined]
      From worker 6:	is_known_call at .\compiler/ssair\passes.jl:5
      From worker 6:	sroa_pass! at .\compiler/ssair\passes.jl:1247
      From worker 6:	run_passes_ipo_safe at .\compiler\optimize.jl:994
      From worker 6:	run_passes_ipo_saf

Worker 17 terminated.
Unhandled Task ERROR: IOError: read: connection reset by peer (ECONNRESET)
Stacktrace:
 [1] wait_readnb(x::Sockets.TCPSocket, nb::Int64)
   @ Base .\stream.jl:410
 [2] (::Base.var"#wait_locked#831")(s::Sockets.TCPSocket, buf::IOBuffer, nb::Int64)
   @ Base .\stream.jl:925
 [3] readbytes!(s::Sockets.TCPSocket, a::Vector{UInt8}, nb::Int64)
   @ Base .\stream.jl:931
 [4] read
   @ .\io.jl:1176 [inlined]
 [5] process_hdr(s::Sockets.TCPSocket, validate_cookie::Bool)
   @ Distributed C:\Users\NizarUribe\.julia\juliaup\julia-1.11.0+0.x64.w64.mingw32\share\julia\stdlib\v1.11\Distributed\src\process_messages.jl:274
 [6] message_handler_loop(r_stream::Sockets.TCPSocket, w_stream::Sockets.TCPSocket, incoming::Bool)
   @ Distributed C:\Users\NizarUribe\.julia\juliaup\julia-1.11.0+0.x64.w64.mingw32\share\julia\stdlib\v1.11\Distributed\src\process_messages.jl:158
 [7] process_tcp_streams(r_stream::Sockets.TCPSocket, w_stream::Sockets.TCPSocket, incoming::Bool)
   @ Distributed 

CompositeException: TaskFailedException

    nested task error: worker did not connect within 60.0 seconds
    Stacktrace:
     [1] error(s::String)
       @ Base .\error.jl:35
     [2] create_worker(manager::Distributed.LocalManager, wconfig::WorkerConfig)
       @ Distributed C:\Users\NizarUribe\.julia\juliaup\julia-1.11.0+0.x64.w64.mingw32\share\julia\stdlib\v1.11\Distributed\src\cluster.jl:691
     [3] setup_launched_worker(manager::Distributed.LocalManager, wconfig::WorkerConfig, launched_q::Vector{Int64})
       @ Distributed C:\Users\NizarUribe\.julia\juliaup\julia-1.11.0+0.x64.w64.mingw32\share\julia\stdlib\v1.11\Distributed\src\cluster.jl:545
     [4] (::Distributed.var"#45#48"{Distributed.LocalManager, Vector{Int64}, WorkerConfig})()
       @ Distributed C:\Users\NizarUribe\.julia\juliaup\julia-1.11.0+0.x64.w64.mingw32\share\julia\stdlib\v1.11\Distributed\src\cluster.jl:501

In [10]:
using CSV
using DataFrames
using Base.Threads
using Printf

# Cargar solo las columnas necesarias en `movies.csv` y mapear `movieId` a `genres`
movies = CSV.File("movies.csv") |> DataFrame
movies_dict = Dict(movies.movieId .=> movies.genres)  # Mapeo `movieId` a `genres`

# Función para contar calificaciones por género en una sola parte del archivo
function count_ratings_by_genre_part(ratings, movies_dict)
    genre_count = Dict{String, Int}()
    for row in eachrow(ratings)
        genres = get(movies_dict, row.movieId, "")
        if !isempty(genres)
            for genre in split(genres, "|")
                genre_count[genre] = get(genre_count, genre, 0) + 1
            end
        end
    end
    return genre_count
end

# Función para cargar y contar cada archivo CSV en paralelo usando `Threads`
function load_and_count(i, movies_dict)
    ratings = CSV.read("ratings_part_$i.csv", DataFrame; select = [:movieId])  # Solo carga `movieId`
    count_ratings_by_genre_part(ratings, movies_dict)
end

# Arreglo de resultados para cada hilo
results = Vector{Dict{String, Int}}(undef, 10)

# Procesamiento en paralelo
Threads.@threads for i in 1:10
    results[i] = load_and_count(i, movies_dict)
end

# Combinar resultados de cada hilo en `final_result`
final_result = Dict{String, Int}()
for result in results
    for (genre, count) in result
        final_result[genre] = get(final_result, genre, 0) + count
    end
end

# Función para formatear números con comas
function format_with_commas(n::Int)
    s = string(n)
    len = length(s)
    if len <= 3 return s end
    result = ""
    for (i, digit) in enumerate(reverse(s))
        if i > 1 && (i - 1) % 3 == 0
            result *= ","
        end
        result *= digit
    end
    return reverse(result)
end

# Imprimir resultados
for (genre, count) in final_result
    formatted_count = format_with_commas(count)
    println("$genre: \t $formatted_count calificaciones")
end


War: 	 1,267,346 calificaciones
Documentary: 	 322,449 calificaciones
Action: 	 7,446,918 calificaciones
Adventure: 	 5,832,424 calificaciones
Mystery: 	 2,010,995 calificaciones
Children: 	 2,124,258 calificaciones
Thriller: 	 6,763,272 calificaciones
Sci-Fi: 	 4,325,740 calificaciones
Comedy: 	 8,926,230 calificaciones
(no genres listed): 	 26,627 calificaciones
IMAX: 	 1,063,279 calificaciones
Fantasy: 	 2,831,585 calificaciones
Drama: 	 10,962,833 calificaciones
Animation: 	 1,630,987 calificaciones
Western: 	 483,731 calificaciones
Horror: 	 1,892,183 calificaciones
Romance: 	 4,497,291 calificaciones
Film-Noir: 	 247,227 calificaciones
Crime: 	 4,190,259 calificaciones
Musical: 	 964,252 calificaciones


In [12]:
using CSV
using DataFrames
using Base.Threads
using Printf

# Cargar el archivo `movies.csv` y mapear `movieId` a `genres`
movies = CSV.File("movies.csv") |> DataFrame
movies_dict = Dict(movies.movieId .=> movies.genres)

# Función para contar y sumar calificaciones por género
function count_and_sum_ratings_by_genre(ratings, movies_dict)
    genre_count = Dict{String, Int}()
    genre_sum = Dict{String, Float64}()
    
    for row in eachrow(ratings)
        genres = get(movies_dict, row.movieId, "")
        if !isempty(genres)
            for genre in split(genres, "|")
                genre_count[genre] = get(genre_count, genre, 0) + 1
                genre_sum[genre] = get(genre_sum, genre, 0.0) + row.rating
            end
        end
    end
    return genre_count, genre_sum
end

# Función para cargar y procesar cada archivo CSV en paralelo
function load_count_and_sum(i, movies_dict)
    ratings = CSV.read("ratings_part_$i.csv", DataFrame; select = [:movieId, :rating])
    count_and_sum_ratings_by_genre(ratings, movies_dict)
end

# Arreglos para almacenar resultados de cada hilo
results_count = Vector{Dict{String, Int}}(undef, 10)
results_sum = Vector{Dict{String, Float64}}(undef, 10)

# Procesamiento en paralelo con Threads
Threads.@threads for i in 1:10
    results_count[i], results_sum[i] = load_count_and_sum(i, movies_dict)
end

# Combinar los resultados de todos los hilos
final_count = Dict{String, Int}()
final_sum = Dict{String, Float64}()

for i in 1:10
    for (genre, count) in results_count[i]
        final_count[genre] = get(final_count, genre, 0) + count
    end
    for (genre, sum_rating) in results_sum[i]
        final_sum[genre] = get(final_sum, genre, 0.0) + sum_rating
    end
end

# Calcular el promedio y almacenar los resultados en un DataFrame
results_df = DataFrame(Genre=String[], Count=Int[], AverageRating=Float64[])

for (genre, count) in final_count
    avg_rating = final_sum[genre] / count
    push!(results_df, (Genre=genre, Count=count, AverageRating=avg_rating))
end

# Guardar los resultados en un archivo CSV
CSV.write("genre_ratings_summary.csv", results_df)

println("Archivo 'genre_ratings_summary.csv' creado con éxito.")


SystemError: SystemError: opening file "genre_ratings_summary.csv": Permission denied